In [184]:
source('utils.r')
source('selinf_functions.r')
source('metrics.r')
source('data_generator.r')
source('cov_matrix.r')
library('lmmlasso')

## Generating the data
In this first attempt at using the minMod option for Selective Inference, we try and use a simpler model, with only 1 random variable other than the intercept.

In [185]:
set.seed(2)

n_subjects= 25
n_observations = 4
n = n_subjects * n_observations
p = 100
q <- 1
SNR = 4
prop_relevant = 0.1

data <- data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant, rho=0.2)
X <- data$X
Z <- data$Z
subjects <- data$subjects
y <- data$y
beta <- data$beta
sd <- data$sd

# fixing lambda
lambda = max(abs(t(X)%*%y))/2

In [186]:
subj_cov_list <- list()
for(i in 1:n_subjects){subj_cov_list[[i]] <- Z[subjects==i,]%*%t(Z[subjects==i,])+sd^2*diag(n_observations)}
true_cov <- bdiag(subj_cov_list)

## Old variance estimate

In [187]:
fixed_form = as.formula(
    paste("y ~ 1 +",paste("X", 1:p, sep="", collapse='+'),sep="")
  )

rnd_form = list(subjects =~ 1 + X1)


selFun <- function(y) selFun_fixed_lambda(X, subjects, y, fixed_form, rand_form, lambda)

sel <- selFun(y)
sel_vec <- sel$vec
sel_names <- sel$names
print(metrics(sel_vec,c(TRUE,beta!=0)))

$tpr
[1] 0.7272727

$fdr
[1] 0.2



In [188]:
## Adding Selective Inference
# Now we can define the function checking the congruency
# with the original selection
checkFun <- function(yb){

  all(selFun(yb)$vec == sel_vec)

}

sel_form = as.formula(
  paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (1 + X1|subjects)")
)

# control <- lmerControl(
#     check.nobs.vs.rankZ = "ignore",
#     check.nobs.vs.nlev = "ignore",
#     check.nlev.gtreq.5 = "ignore",
#     check.nlev.gtr.1 = "ignore",
#     check.nobs.vs.nRE= "ignore",
# )


final_model = lmer(formula = sel_form ,data=data.frame(X, subjects, y))

In [189]:
final_model

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: sel_form
   Data: data.frame(X, subjects, y)
REML criterion at convergence: 424.3323
Random effects:
 Groups   Name        Std.Dev. Corr 
 subjects (Intercept) 0.9724        
          X1          1.2601   -0.47
 Residual             1.5960        
Number of obs: 100, groups:  subjects, 25
Fixed Effects:
(Intercept)           X1           X3           X4           X5           X6  
    0.93807     -0.95927      1.15492      1.20670      1.00849      1.03741  
         X7          X10          X26          X32  
   -1.05936      0.92338      0.01746      0.08861  

In [191]:
res <- mocasin(final_model, this_y = y, conditional = FALSE, varForSampling = 'minMod',
               checkFun = checkFun, nrSamples = 50)
               
# create a boolean vector for the ones selected controlling fdr level with BH procedure

sel_with_selinf <- selection_with_selinf(res, sel_vec, fdr_level = 0.1)
metrics(sel_with_selinf,c(1,beta!=0))

Warning message in diag(n/nrsubj) * sigma^2 + tau^2:
"longer object length is not a multiple of shorter object length"


Computing inference for variable (location)  1 

  |======================================================================| 100%


Computing inference for variable (location)  2 

  |======================================================================| 100%


Computing inference for variable (location)  3 

  |======================================================================| 100%


Computing inference for variable (location)  4 

  |======================================================================| 100%


Computing inference for variable (location)  5 

  |======================================================================| 100%


Computing inference for variable (location)  6 

  |======================================================================| 100%


Computing inference for variable (location)  7 

  |======================================================================| 100%


Computing inference for variable (location)  8 

  |===============================

$tpr
[1] 0.5454545

$fdr
[1] 0.1428571

In [192]:
ci_length(res)

[1] 8.381879

## New variance estimate

In [193]:
source('minMod_modif.r')

In [198]:
res <- mocasin(final_model, this_y = y, conditional = FALSE, varForSampling = 'minMod',
               checkFun = checkFun, nrSamples = 50)
# create a boolean vector for the ones selected controlling fdr level with BH procedure

sel_with_selinf <- selection_with_selinf(res, sel_vec, fdr_level = 0.1)
metrics(sel_with_selinf,c(1,beta!=0))

Computing inference for variable (location)  1 

  |======================================================================| 100%


Computing inference for variable (location)  2 

  |======================================================================| 100%


Computing inference for variable (location)  3 

  |======================================================================| 100%


Computing inference for variable (location)  4 

  |======================================================================| 100%


Computing inference for variable (location)  5 

  |======================================================================| 100%


Computing inference for variable (location)  6 

  |======================================================================| 100%


Computing inference for variable (location)  7 

  |======================================================================| 100%


Computing inference for variable (location)  8 

  |===============================

$tpr
[1] 0.4545455

$fdr
[1] 0

In [199]:
ci_length(res)

[1] 8.894143